# Freyberg USG Model PEST setup example
Herein, we will show users how to use pyEMU to setup a groundwater model for use in pest. Except using the Unstructured Grid (usg) version of MODFLOW. We will cover the following topics:
- setup pilot points as parameters, including 1st-order tikhonov regularization
- setup other model inputs as parameters
- setup simulated water levels as observations
- setup simulated water budget components as observations (or forecasts)
- create a pest control file and adjust observation weights to balance the objective function

Note that, in addition to `pyemu`, this notebook relies on `flopy`. `flopy` can be obtained (along with installation instructions) at https://github.com/modflowpy/flopy.


In [1]:
%matplotlib inline
import os
import shutil
import platform
import numpy as np
import pandas as pd
from matplotlib.patches import Rectangle as rect
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", 
    message="ModflowDis.sr is deprecated. use Modflow.sr")
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
newparams = {'legend.fontsize':10, 'axes.labelsize':10,
             'xtick.labelsize':10, 'ytick.labelsize':10,
             'font.family':'Univers 57 Condensed', 
             'pdf.fonttype':42}
plt.rcParams.update(newparams)
import pyemu

## Model background
This example is based on the synthetic classroom model of Freyberg(1988).  The  model is a 2-dimensional MODFLOW model with 3 layers,  40 rows, and 20 columns.  The model has 2 stress periods: an initial steady-state stress period used for calibration, and a 5-year transient stress period.  The calibration period uses the recharge and well flux of Freyberg(1988); the last stress period use 25% less recharge and 25% more pumping to represent future conditions for a forecast period.

This model has been modified using Gridgen to include a quadtree mesh at the location of the river.

Freyberg, David L. "AN EXERCISE IN GROUND‐WATER MODEL CALIBRATION AND PREDICTION." Groundwater 26.3 (1988): 350-360.

In [27]:
#load the existing model and save it in a new dir and make sure it runs
import flopy
model_ws = os.path.join("freyberg_usg")
ml = flopy.modflow.Modflow.load("freyberg.usg.nam",model_ws=model_ws,verbose=False,version='mfusg',forgive=True,check=False)
ml.exe_name = "mfusg"
# ml.model_ws = "temp"
EXE_DIR = os.path.join("..","bin")
if "window" in platform.platform().lower():
    EXE_DIR = os.path.join(EXE_DIR,"win")
elif "darwin" in platform.platform().lower():
    EXE_DIR = os.path.join(EXE_DIR,"mac")
else:
    EXE_DIR = os.path.join(EXE_DIR,"linux")

[shutil.copy2(os.path.join(EXE_DIR,f),os.path.join("temp",f)) for f in os.listdir(EXE_DIR)]

ml.external_path = "."
ml.change_model_ws('temp',reset_external=True)
ml.write_input()
ml.run_model()


changing model workspace...
   temp
Util2d:nodelay: resetting 'how' to external
Util2d:top: resetting 'how' to external
Util2d:top: resetting 'how' to external
Util2d:top: resetting 'how' to external
Util2d:btk: resetting 'how' to external
Util2d:btk: resetting 'how' to external
Util2d:btk: resetting 'how' to external
Util2d:ak: resetting 'how' to external
Util2d:ak: resetting 'how' to external
Util2d:ak: resetting 'how' to external
Util2d:iac: resetting 'how' to external
Util2d:ja: resetting 'how' to external
Util2d:cl12: resetting 'how' to external
Util2d:fahl: resetting 'how' to external
Util2d:ibound_layer_0: resetting 'how' to external
Util2d:ibound_layer_1: resetting 'how' to external
Util2d:ibound_layer_2: resetting 'how' to external
Util2d:strt_layer_0: resetting 'how' to external
Util2d:strt_layer_1: resetting 'how' to external
Util2d:strt_layer_2: resetting 'how' to external
Util2d:hk: resetting 'how' to external
Util2d:vk: resetting 'how' to external
Util2d:ss: resetting 'h

(True, [])

In [3]:
def hdobj2data(hdsobj): 
    # convert usg hdsobj to array of shape (nper, nnodes)
    hds = []
    kstpkpers = hdsobj.get_kstpkper()
    for kstpkper in kstpkpers:
        data = hdsobj.get_data(kstpkper=kstpkper)
        fdata = []
        for lay in range(len(data)):
            fdata += data[lay].tolist()
        hds.append(fdata)

    return np.array(hds)

In [4]:
node_df = pd.read_csv(os.path.join("Freyberg","misc","obs_nodes.dat"),delim_whitespace=True)
hdsobj = flopy.utils.HeadUFile(os.path.join(ml.model_ws,"freyberg.usg.hds"))
hds = hdobj2data(hdsobj)
nper,nnodes = hds.shape
(nper,nnodes)

(25, 4497)

In [5]:
data = []
for i, dfrow in node_df.iterrows():
    name, node = dfrow['name'], dfrow['node']
    
    r = np.random.randn(nper) #add some random noise to the observations
    for sp in range(nper):
        
        hd = hds[sp,node-1]
        rhd = r[sp] + hd #add some random noise to the observations
        
        data.append([hd,rhd,name,node,sp])


obs_df = pd.DataFrame(data,columns=['simval','obsval','name','node','sp'])
obs_df.to_csv(os.path.join(ml.model_ws,'obs.csv'),index=False)
obs_df

,simval,obsval,name,node,sp
0,34.552040,35.350417,obs_01,107,0
1,34.601082,34.628617,obs_01,107,1
2,34.646980,34.000549,obs_01,107,2
3,34.673031,37.139065,obs_01,107,3
4,34.666801,36.141500,obs_01,107,4
...,...,...,...,...,...
295,33.163342,33.387071,obs_13,1348,20
296,33.149437,33.992372,obs_13,1348,21
297,33.196201,34.146785,obs_13,1348,22
298,33.287243,33.008601,obs_13,1348,23


In [32]:
v = pyemu.geostats.ExpVario(contribution=1.0,a=1000)
gs = pyemu.geostats.GeoStruct(variograms=v)

In [30]:
template_ws = 'template'
pf = pyemu.utils.PstFrom('temp',template_ws, longnames=True,remove_existing=True,
                             zero_based=False,spatial_reference=gsf.get_node_coordinates(zero_based=True))

2021-06-04 14:53:57.566427 starting: opening PstFrom.log for logging
2021-06-04 14:53:57.568069 starting PstFrom process
2021-06-04 14:53:57.570112 dictionary-based spatial reference detected...
2021-06-04 14:53:57.571071 starting: setting up dirs
2021-06-04 14:53:57.575975 starting: copying original_d 'temp' to new_d 'template'
2021-06-04 14:54:00.196577 finished: copying original_d 'temp' to new_d 'template' took: 0:00:02.620602
2021-06-04 14:54:00.202586 finished: setting up dirs took: 0:00:02.631515


# Parameters

## pilot points

Here we will import pilot point locations from a csv


In [6]:
pp_df = pd.read_csv(os.path.join("Freyberg","misc","pp_usg.csv"))
pp_df['xy'] = pp_df.apply(lambda i: (i['x'],i['y']),axis=1)
pp_df.head()

,name,x,y,xy
0,pp_0000,620116.4,3372795.9,"(620116.4, 3372795.9)"
1,pp_0001,620866.4,3372795.9,"(620866.4, 3372795.9)"
2,pp_0002,621616.4,3372795.9,"(621616.4, 3372795.9)"
3,pp_0003,622366.4,3372795.9,"(622366.4, 3372795.9)"
4,pp_0004,623116.4,3372795.9,"(623116.4, 3372795.9)"


### Use the GSF to make a Spatial Refrence structure  

In [13]:
gsf = pyemu.gw_utils.GsfReader(os.path.join(model_ws,"freyberg.usg.gsf"))
gsf_df = gsf.get_node_data()
gsf_df["xy"] = gsf_df.apply(lambda i: (i['x'],i['y']),axis=1)
gsf_df.head()

,node,x,y,z,layer,numverts,vertidx,xy
0,1,619866.4,3373045.9,36.95120,1,8,"[1, 37, 38, 2, 1756, 1792, 1793, 1757]","(619866.4, 3373045.9)"
1,2,620116.4,3373045.9,36.91541,1,8,"[2, 38, 39, 3, 1757, 1793, 1794, 1758]","(620116.4, 3373045.9)"
2,3,620366.4,3373045.9,36.84000,1,8,"[3, 39, 40, 4, 1758, 1794, 1795, 1759]","(620366.4, 3373045.9)"
3,4,620616.4,3373045.9,36.72716,1,8,"[4, 40, 41, 5, 1759, 1795, 1796, 1760]","(620616.4, 3373045.9)"
4,5,620866.4,3373045.9,36.60465,1,8,"[5, 41, 42, 6, 1760, 1796, 1797, 1761]","(620866.4, 3373045.9)"


### setup pilot point locations

first specify what pilot point names we want to use for each model layer (counting from 0).  Here we will setup pilot points for ``hk``, ``sy`` and ``rech``.  The ``rech`` pilot points will be used as a single multiplier array for all stress periods to account for potential spatial bias in recharge.

### setup sr dict (location for each node)
for usg, we need to do some trickery to support the unstructured by layers concept
this is just for array-based parameters, list-based pars are g2g because they have an index

In [15]:
nlay = ml.nlay

sr_dict_by_layer = {}
for lay in range(nlay):
    df_lay = gsf_df.loc[gsf_df['layer'] == lay+1]
    df_lay.loc[:,"node"] = df_lay['node'] - df_lay['node'].min()
    srd = {n:xy for n,xy in zip(df_lay.node.values,df_lay.xy.values)}
    sr_dict_by_layer[lay+1] = srd
    
    

C:\Users\rkushnereit\Miniconda3\envs\geopy38\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\rkushnereit\Miniconda3\envs\geopy38\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\rkushnereit\Miniconda3\envs\geopy38\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying 

In [33]:
for lay in range(nlay):
    pf.add_parameters(os.path.join(f"hk_Layer_{lay+1}.ref"), par_type="pilotpoints",
                          par_name_base=f"hk{lay+1}_pp", pp_space=pp_df.copy(),
                          geostruct=gs, spatial_reference=sr_dict_by_layer[lay+1],
                          upper_bound=10.0, lower_bound=0.1)
    
    pf.add_parameters(os.path.join(f"vka{lay+1}.ref"), par_type="pilotpoints", par_name_base=f"vka{lay+1}_pp",pp_space=pp_df.copy(),
                      geostruct=gs,spatial_reference=sr_dict_by_layer[lay+1],
                      upper_bound=10.0,lower_bound=0.1)

    pf.add_parameters(os.path.join(f"ss_Layer_{lay+1}.ref"), par_type="pilotpoints",
                      par_name_base=f"ss{lay+1}_pp", pp_space=pp_df.copy(),
                      geostruct=gs, spatial_reference=sr_dict_by_layer[lay+1],
                      upper_bound=10.0, lower_bound=0.1)

2021-06-04 14:55:03.765756 starting: adding pilotpoints type multiplier style parameters for file(s) ['hk_Layer_1.ref']
2021-06-04 14:55:03.766736 WARNING: 0) Inconsistency between geostruct transform and partrans.
2021-06-04 14:55:03.766736 WARNING: 1) Setting geostruct transform to log
2021-06-04 14:55:03.766736 WARNING: -) Better to pass an appropriately transformed geostruct
2021-06-04 14:55:03.767735 starting: loading array template\hk_Layer_1.ref
2021-06-04 14:55:03.778730 finished: loading array template\hk_Layer_1.ref took: 0:00:00.010995
2021-06-04 14:55:03.779731 loaded array 'temp\hk_Layer_1.ref' of shape (1, 1499)
2021-06-04 14:55:03.794677 starting: writing array-based template file 'template\hk1_pp_inst0_pilotpoints.csv.tpl'
2021-06-04 14:55:03.795409 starting: setting up pilot point parameters
2021-06-04 14:55:03.795409 WARNING: 'zone' col not found in pp dataframe, adding generic zone
2021-06-04 14:55:03.817218 201 pilot point parameters created
2021-06-04 14:55:03.8178

C:\Users\rkushnereit\Miniconda3\envs\geopy38\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


took 14.011268 seconds
OrdinaryKrige.to_grid_factors_file(): spatial_reference attr is None, assuming unstructured grid
2021-06-04 14:55:17.995866 finished: calculating factors for pargp=hk1_pp_inst:0 took: 0:00:14.151277
2021-06-04 14:55:17.996789 starting: writing array-based template file 'template\hk1_pp_inst0pp.dat.tpl'
2021-06-04 14:55:18.004787 finished: adding pilotpoints type multiplier style parameters for file(s) ['hk_Layer_1.ref'] took: 0:00:14.239031
2021-06-04 14:55:18.006413 starting: adding pilotpoints type multiplier style parameters for file(s) ['vka1.ref']
2021-06-04 14:55:18.007763 starting: loading array template\vka1.ref
2021-06-04 14:55:18.016503 finished: loading array template\vka1.ref took: 0:00:00.008740
2021-06-04 14:55:18.017432 loaded array 'temp\vka1.ref' of shape (1, 1499)
2021-06-04 14:55:18.030279 starting: writing array-based template file 'template\vka1_pp_inst0_pilotpoints.csv.tpl'
2021-06-04 14:55:18.030761 starting: setting up pilot point paramete